## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
gcc6_0_32 = pd.read_csv("../data/binutils_gcc3~9_op0~4_exec_csv/"+'gcc6'+"_0_32_exec.csv", index_col=0)

# 형태 출력
print(gcc6_0_32.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
gcc6_0_32.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
gcc6_0_32.head()

(13936744, 2)
reset_index 완료
input data shape


,bin,label
0,53,1
1,83,0
2,ec,0
3,08,0
4,e8,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 

# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(gcc6_0_32['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(gcc6_0_32['label'].value_counts())

256
0    13892354
1       44390
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
######################## 
idx = gcc6_0_32[gcc6_0_32['label']==1].index  # 407, 474 ...
ls = list(idx)

# 최종 뽑을 행에 대한 index
ls_idx = [] 
left_idx, right_idx = 0, 32 # 3개씩

# 6gram
for k in range(left_idx, right_idx):
    ls_idx.extend(list(idx + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx = list(filter(lambda x: x<len(gcc6_0_32), ls_idx))
print(len(ls_idx))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub = len(ls_idx)%(right_idx)
print('나머지', sub)

ls_idx = ls_idx[:len(ls_idx)-sub]
print('최종 길이', len(ls_idx))

print('gcc6_0_32', len(ls_idx))

# loc 로 수정필요
gcc6_0_32_Ngram = gcc6_0_32.loc[ls_idx,:].copy()

1420468
나머지 20
최종 길이 1420448
gcc6_0_32 1420448


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성
# random index
random_idx = np.random.randint(len(gcc6_0_32)-right_idx)

# 목표치
goal = len(gcc6_0_32_Ngram)/right_idx
count=0

d = pd.DataFrame(columns=gcc6_0_32.columns)

# goal 에 도달할 때까지
while count!=goal:
    
    # 진행상황 살펴보기 위함
    if count%1000==0:
        print(count, end=' ')
        
    # 랜덤 N 바이트씩 뽑음
    df = gcc6_0_32[random_idx:random_idx+right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df['label']:
        d = pd.concat([d, df])
        count+=1

print('완료')

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 완료


## (5) False Data + True Data 합치기

In [5]:
ff = pd.DataFrame(columns=gcc6_0_32.columns)

for i in range(0, int(len(d)/right_idx)):
    ff = pd.concat([ff, gcc6_0_32[i*right_idx:(i+1)*right_idx], d[i*right_idx:(i+1)*right_idx]])
    
    if i%1000==0:
        print(i, end=' ')
ff.shape

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 40000 41000 42000 43000 44000 

(2840896, 2)

## (6) one hot encoding

In [6]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
gcc6_0_32_onehot_Ngram = pd.get_dummies(ff['bin'])
gcc6_0_32_onehot_Ngram = pd.concat([ff['label'], gcc6_0_32_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(gcc6_0_32_onehot_Ngram.shape)

원핫인코딩완료
(2840896, 257)


In [7]:
# 훈련 데이터, 훈련 라벨
x_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram.iloc[:,1:].to_numpy()
y_gcc6_0_32_3 = gcc6_0_32_onehot_Ngram['label'].to_numpy()
print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

x_gcc6_0_32_3 = x_gcc6_0_32_3.reshape(-1, right_idx, x_gcc6_0_32_3.shape[1])
y_gcc6_0_32_3 = y_gcc6_0_32_3.reshape(-1, right_idx, 1)

print(x_gcc6_0_32_3.shape, y_gcc6_0_32_3.shape)

(2840896, 256) (2840896,)
(88778, 32, 256) (88778, 32, 1)


## (7) 모델

In [8]:
# (10) 양방향 LSTM 모델링 작업
from keras.models import Model, Sequential
from keras.layers import SimpleRNN, Input, Dense, LSTM
from keras.layers import Bidirectional, TimeDistributed

# 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = Bidirectional(LSTM(16, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = TimeDistributed(Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

model1 = Model(xInput, xOutput)
model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model1.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 32)            34944     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 32, 1)             33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


## (8) 학습 - 10 KFold

In [9]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_gcc6_0_32_3, y_gcc6_0_32_3):
    print('======Training stage======')
    model1.fit(x_gcc6_0_32_3[train],
               y_gcc6_0_32_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_gcc6_0_32_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_gcc6_0_32_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 14. 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

======Training stage======
Epoch 1/10
79900/79900 [==============================] - 45s 559us/step - loss: 0.0109 - accuracy: 0.9988
Epoch 2/10
79900/79900 [==============================] - 44s 556us/step - loss: 0.0027 - accuracy: 0.9992
Epoch 3/10
79900/79900 [==============================] - 46s 575us/step - loss: 0.0024 - accuracy: 0.9993
Epoch 4/10
79900/79900 [==============================] - 46s 577us/step - loss: 0.0022 - accuracy: 0.9993
Epoch 5/10
79900/79900 [==============================] - 46s 574us/step - loss: 0.0021 - accuracy: 0.9993
Epoch 6/10
79900/79900 [==============================] - 46s 572us/step - loss: 0.0020 - accuracy: 0.9994
Epoch 7/10
79900/79900 [==============================] - 46s 574us/step - loss: 0.0020 - accuracy: 0.9994
Epoch 8/10
79900/79900 [==============================] - 45s 567us/step - loss: 0.0019 - accuracy: 0.9994
Epoch 9/10
79900/79900 [==============================] - 45s 563us/step - loss: 0.0018 - accuracy: 0.9994
Epoch 10/1

accuracy_score 0.9993171322369904
recall_score 0.6203966005665722
precision_score 0.7849462365591398
f1_score 0.6930379746835442
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 44s 544us/step - loss: 0.0018 - accuracy: 0.9994
Epoch 2/10
79900/79900 [==============================] - 44s 550us/step - loss: 0.0018 - accuracy: 0.9994
Epoch 3/10
79900/79900 [==============================] - 45s 559us/step - loss: 0.0017 - accuracy: 0.9995
Epoch 4/10
79900/79900 [==============================] - 44s 557us/step - loss: 0.0017 - accuracy: 0.9995
Epoch 5/10
79900/79900 [==============================] - 45s 558us/step - loss: 0.0017 - accuracy: 0.9995
Epoch 6/10
79900/79900 [==============================] - 45s 562us/step - loss: 0.0016 - accuracy: 0.9995
Epoch 7/10
79900/79900 [==============================] - 45s 564us/step - loss: 0.0016 - accuracy: 0.9995
Epoch 8/10
79900/79900 [==============================] - 45s 562us/step - loss: 0.0016 - accur

accuracy_score 0.9994755293985131
recall_score 0.7713414634146342
precision_score 0.7737003058103975
f1_score 0.7725190839694657
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 45s 558us/step - loss: 0.0015 - accuracy: 0.9995
Epoch 2/10
79900/79900 [==============================] - 45s 563us/step - loss: 0.0015 - accuracy: 0.9995
Epoch 3/10
79900/79900 [==============================] - 45s 564us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 4/10
79900/79900 [==============================] - 45s 568us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 5/10
79900/79900 [==============================] - 45s 568us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 6/10
79900/79900 [==============================] - 46s 571us/step - loss: 0.0014 - accuracy: 0.9996
Epoch 7/10
79900/79900 [==============================] - 46s 570us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 8/10
79900/79900 [==============================] - 46s 573us/step - loss: 0.0013 - accur

accuracy_score 0.9995177686415859
recall_score 0.7941176470588235
precision_score 0.8011869436201781
f1_score 0.7976366322008863
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 45s 560us/step - loss: 0.0013 - accuracy: 0.9996
Epoch 2/10
79900/79900 [==============================] - 45s 563us/step - loss: 0.0012 - accuracy: 0.9996
Epoch 3/10
79900/79900 [==============================] - 45s 560us/step - loss: 0.0012 - accuracy: 0.9996
Epoch 4/10
79900/79900 [==============================] - 45s 561us/step - loss: 0.0012 - accuracy: 0.9997
Epoch 5/10
79900/79900 [==============================] - 42s 531us/step - loss: 0.0011 - accuracy: 0.9997
Epoch 6/10
79900/79900 [==============================] - 40s 499us/step - loss: 0.0011 - accuracy: 0.9997
Epoch 7/10
79900/79900 [==============================] - 40s 498us/step - loss: 0.0011 - accuracy: 0.9997s - loss: 0.0011 - accura
Epoch 8/10
79900/79900 [==============================] - 39s 492us/st

accuracy_score 0.9995353683261996
recall_score 0.7530120481927711
precision_score 0.8333333333333334
f1_score 0.7911392405063291
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 39s 492us/step - loss: 0.0010 - accuracy: 0.9997
Epoch 2/10
79900/79900 [==============================] - 39s 489us/step - loss: 0.0010 - accuracy: 0.9997
Epoch 3/10
79900/79900 [==============================] - 39s 493us/step - loss: 9.7195e-04 - accuracy: 0.9997s - los
Epoch 4/10
79900/79900 [==============================] - 39s 494us/step - loss: 9.5360e-04 - accuracy: 0.9997
Epoch 5/10
79900/79900 [==============================] - 40s 495us/step - loss: 9.3048e-04 - accuracy: 0.9997
Epoch 6/10
79900/79900 [==============================] - 39s 494us/step - loss: 9.1642e-04 - accuracy: 0.9997s - l
Epoch 7/10
79900/79900 [==============================] - 39s 494us/step - loss: 8.8567e-04 - accuracy: 0.9997
Epoch 8/10
79900/79900 [==============================] - 40s 4

...

accuracy_score 0.9995987271908088
recall_score 0.7850746268656716
precision_score 0.8622950819672132
f1_score 0.8218750000000001
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 40s 503us/step - loss: 8.0277e-04 - accuracy: 0.9998
Epoch 2/10
79900/79900 [==============================] - 40s 502us/step - loss: 7.9450e-04 - accuracy: 0.9998
Epoch 3/10
79900/79900 [==============================] - 40s 502us/step - loss: 7.4444e-04 - accuracy: 0.9998
Epoch 4/10
79900/79900 [==============================] - 40s 504us/step - loss: 7.3608e-04 - accuracy: 0.9998
Epoch 5/10
79900/79900 [==============================] - 40s 502us/step - loss: 7.1076e-04 - accuracy: 0.9998
Epoch 6/10
79900/79900 [==============================] - 40s 503us/step - loss: 6.9351e-04 - accuracy: 0.9998
Epoch 7/10
79900/79900 [==============================] - 40s 503us/step - loss: 6.7572e-04 - accuracy: 0.9998
Epoch 8/10
79900/79900 [==============================] - 40s 502us

accuracy_score 0.9996726458661861
recall_score 0.8162650602409639
precision_score 0.8943894389438944
f1_score 0.8535433070866143
======Training stage======
Epoch 1/10
79900/79900 [==============================] - 40s 505us/step - loss: 6.9732e-04 - accuracy: 0.9998
Epoch 2/10
79900/79900 [==============================] - 40s 505us/step - loss: 6.6112e-04 - accuracy: 0.9998
Epoch 3/10
79900/79900 [==============================] - 40s 504us/step - loss: 6.5027e-04 - accuracy: 0.9998s - loss: 6.5158e-04 - accura
Epoch 4/10
79900/79900 [==============================] - 40s 505us/step - loss: 6.3157e-04 - accuracy: 0.9998
Epoch 5/10
79900/79900 [==============================] - 40s 503us/step - loss: 6.0712e-04 - accuracy: 0.9998
Epoch 6/10
79900/79900 [==============================] - 40s 503us/step - loss: 5.9916e-04 - accuracy: 0.9998
Epoch 7/10
79900/79900 [==============================] - 40s 505us/step - loss: 5.8531e-04 - accuracy: 0.9998
Epoch 8/10
79900/79900 [==============

accuracy_score 0.999648006307727
recall_score 0.8270893371757925
precision_score 0.8776758409785933
f1_score 0.8516320474777448
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 44s 546us/step - loss: 6.5769e-04 - accuracy: 0.9998
Epoch 2/10
79901/79901 [==============================] - 44s 545us/step - loss: 6.0936e-04 - accuracy: 0.9998
Epoch 3/10
79901/79901 [==============================] - 44s 545us/step - loss: 6.0430e-04 - accuracy: 0.9998
Epoch 4/10
79901/79901 [==============================] - 44s 546us/step - loss: 5.7700e-04 - accuracy: 0.9998
Epoch 5/10
79901/79901 [==============================] - 44s 545us/step - loss: 5.4750e-04 - accuracy: 0.9999
Epoch 6/10
79901/79901 [==============================] - 44s 544us/step - loss: 5.5154e-04 - accuracy: 0.9999
Epoch 7/10
79901/79901 [==============================] - 44s 545us/step - loss: 5.2164e-04 - accuracy: 0.9999
Epoch 8/10
79901/79901 [==============================] - 44s 545us/

accuracy_score 0.9996761293229695
recall_score 0.8493506493506493
precision_score 0.9058171745152355
f1_score 0.8766756032171581
======Training stage======
Epoch 1/10
79901/79901 [==============================] - 44s 546us/step - loss: 5.7997e-04 - accuracy: 0.9998
Epoch 2/10
79901/79901 [==============================] - 44s 546us/step - loss: 5.7639e-04 - accuracy: 0.9998
Epoch 3/10
79901/79901 [==============================] - 44s 545us/step - loss: 5.4381e-04 - accuracy: 0.9999
Epoch 4/10
79901/79901 [==============================] - 44s 545us/step - loss: 5.3137e-04 - accuracy: 0.9999
Epoch 5/10
79901/79901 [==============================] - 44s 545us/step - loss: 4.9847e-04 - accuracy: 0.9999
Epoch 6/10
79901/79901 [==============================] - 44s 545us/step - loss: 5.0386e-04 - accuracy: 0.9999
Epoch 7/10
79901/79901 [==============================] - 44s 545us/step - loss: 4.9492e-04 - accuracy: 0.9999
Epoch 8/10
79901/79901 [==============================] - 44s 548us

accuracy_score 0.9997042919905373
recall_score 0.8518518518518519
precision_score 0.92
f1_score 0.8846153846153846

K-fold cross validation Accuracy: [0.9993171322369904, 0.9994755293985131, 0.9995177686415859, 0.9995353683261996, 0.9995318483892769, 0.9995987271908088, 0.9996726458661861, 0.999648006307727, 0.9996761293229695, 0.9997042919905373]

K-fold cross validation Recall: [0.6203966005665722, 0.7713414634146342, 0.7941176470588235, 0.7530120481927711, 0.7463976945244957, 0.7850746268656716, 0.8162650602409639, 0.8270893371757925, 0.8493506493506493, 0.8518518518518519]

K-fold cross validation Precision: [0.7849462365591398, 0.7737003058103975, 0.8011869436201781, 0.8333333333333334, 0.8519736842105263, 0.8622950819672132, 0.8943894389438944, 0.8776758409785933, 0.9058171745152355, 0.92]

K-fold cross validation F1-Score: [0.6930379746835442, 0.7725190839694657, 0.7976366322008863, 0.7911392405063291, 0.7956989247311829, 0.8218750000000001, 0.8535433070866143, 0.851632047477744

## (9) 평가지표

In [10]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9995677447670795
10-Fold Cross_validation. Recall : 0.7814896979242225
10-Fold Cross_validation. Precision : 0.8505318039938512
10-Fold Cross_validation. F1-Score : 0.8138373198488311
